<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches_evaluation_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.74-cp310-cp310-linux_x86_64.whl size=61875621 sha256=1c5a798007f77fa71fa88d647372271e6dbdc2568123305fa7117a5c06e58f48
  Stored in directory: /root/.cache/pip/wheels/21/1f/6d/ee16805ea65ed00d89776a8ce82f3631340a43fc41edabed97
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf to /root/.cache/huggingface/hub/tmpgj1buli5
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [01:29<00:00, 85.5MB/s]
./models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.2 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [6]:
### Retrieval Grader

from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser

prompt = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()

In [7]:
import pandas as pd
app1 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach1.json')
len(app1)

64

In [8]:
results_list = []
for i in range(len(app1)):
  question = app1['question'][i]
  doc_txt = app1['context_cosine'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)


llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       4.52 ms /     7 runs   (    0.65 ms per token,  1550.04 tokens per second)
llama_print_timings: prompt eval time =    4537.35 ms /   792 tokens (    5.73 ms per token,   174.55 tokens per second)
llama_print_timings:        eval time =     299.10 ms /     6 runs   (   49.85 ms per token,    20.06 tokens per second)
llama_print_timings:       total time =    5203.59 ms /   798 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       4.86 ms /     7 runs   (    0.69 ms per token,  1439.14 tokens per second)
llama_print_timings: prompt eval time =    3602.06 ms /   613 tokens (    5.88 ms per token,   170.18 tokens per second)
llama_print_timings:        eval time =     229.67 ms /     6 runs   (   38.28 ms per token,    26.12 tokens per second)
llama_print_timings:       total time =    4471.33 ms /   619 

In [10]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app1 = score/len(results)
print(f'Total score for Approach # 1 is {total_score_app1}')

Total score for Approach # 1 is 0.703125


In [11]:
app2 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach2.json')
len(app2)

66

In [12]:
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException

In [13]:
results_list = []
for i in range(len(app2)):
  question = app2['question_name'][i]
  doc_txt = app2['context_cosine_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       2.86 ms /     6 runs   (    0.48 ms per token,  2098.64 tokens per second)
llama_print_timings: prompt eval time =    6101.85 ms /  1083 tokens (    5.63 ms per token,   177.49 tokens per second)
llama_print_timings:        eval time =     204.53 ms /     5 runs   (   40.91 ms per token,    24.45 tokens per second)
llama_print_timings:       total time =    6631.67 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.84 ms /     7 runs   (    0.55 ms per token,  1823.87 tokens per second)
llama_print_timings: prompt eval time =    5147.81 ms /   926 tokens (    5.56 ms per token,   179.88 tokens per second)
llama_print_timings:        eval time =     245.37 ms /     6 runs   (   40.89 ms per token,    24.45 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.38 ms /     6 runs   (    0.56 ms per token,  1775.15 tokens per second)
llama_print_timings: prompt eval time =    3394.67 ms /   629 tokens (    5.40 ms per token,   185.29 tokens per second)
llama_print_timings:        eval time =     195.54 ms /     5 runs   (   39.11 ms per token,    25.57 tokens per second)
llama_print_timings:       total time =    3794.49 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       6.15 ms /     9 runs   (    0.68 ms per token,  1462.94 tokens per second)
llama_print_timings: prompt eval time =    4005.42 ms /   735 tokens (    5.45 ms per token,   183.50 tokens per second)
llama_print_timings:        eval time =     320.76 ms /     8 runs   (   40.10 ms per token,    24.94 tokens per second)
llama_print_timings:       total time =    4633.86 ms /   743 

In [15]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app2 = score/len(results)
print(f'Total score for Approach # 2 is {total_score_app2}')

Total score for Approach # 2 is 0.6615384615384615


In [16]:
app3 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach3.json')
len(app3)

64

In [17]:
results_list = []
for i in range(len(app3)):
  question = app3['question'][i]
  doc_txt = app3['context_ip'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.48 ms /     7 runs   (    0.50 ms per token,  2009.18 tokens per second)
llama_print_timings: prompt eval time =    3582.02 ms /   659 tokens (    5.44 ms per token,   183.97 tokens per second)
llama_print_timings:        eval time =     239.85 ms /     6 runs   (   39.98 ms per token,    25.02 tokens per second)
llama_print_timings:       total time =    4102.66 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.80 ms /     7 runs   (    0.54 ms per token,  1841.14 tokens per second)
llama_print_timings: prompt eval time =    3298.32 ms /   613 tokens (    5.38 ms per token,   185.85 tokens per second)
llama_print_timings:        eval time =     231.97 ms /     6 runs   (   38.66 ms per token,    25.87 tokens per second)
llama_print_timings:       to

In [19]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app3 = score/len(results)
print(f'Total score for Approach # 3 is {total_score_app3}')

Total score for Approach # 3 is 0.703125


In [20]:
app4 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach4.json')
len(app4)

66

In [21]:
results_list = []
for i in range(len(app4)):
  question = app4['question_name'][i]
  doc_txt = app4['context_ip_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.04 ms /     6 runs   (    0.51 ms per token,  1973.04 tokens per second)
llama_print_timings: prompt eval time =    6099.51 ms /  1083 tokens (    5.63 ms per token,   177.56 tokens per second)
llama_print_timings:        eval time =     203.31 ms /     5 runs   (   40.66 ms per token,    24.59 tokens per second)
llama_print_timings:       total time =    6768.43 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       4.30 ms /     7 runs   (    0.61 ms per token,  1627.53 tokens per second)
llama_print_timings: prompt eval time =    5105.96 ms /   926 tokens (    5.51 ms per token,   181.36 tokens per second)
llama_print_timings:        eval time =     243.73 ms /     6 runs   (   40.62 ms per token,    24.62 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.25 ms /     6 runs   (    0.54 ms per token,  1845.02 tokens per second)
llama_print_timings: prompt eval time =    3409.35 ms /   629 tokens (    5.42 ms per token,   184.49 tokens per second)
llama_print_timings:        eval time =     197.71 ms /     5 runs   (   39.54 ms per token,    25.29 tokens per second)
llama_print_timings:       total time =    3866.00 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       4.89 ms /     9 runs   (    0.54 ms per token,  1841.62 tokens per second)
llama_print_timings: prompt eval time =    3988.90 ms /   735 tokens (    5.43 ms per token,   184.26 tokens per second)
llama_print_timings:        eval time =     319.43 ms /     8 runs   (   39.93 ms per token,    25.05 tokens per second)
llama_print_timings:       total time =    4529.35 ms /   743 

In [23]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app4 = score/len(results)
print(f'Total score for Approach # 4 is {total_score_app4}')

Total score for Approach # 4 is 0.6615384615384615


In [24]:
app5 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach5.json')
len(app5)

64

In [25]:
results_list = []
for i in range(len(app5)):
  question = app5['question'][i]
  doc_txt = app5['context_l2'][i]['page_content']
  answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
  results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       6.11 ms /     7 runs   (    0.87 ms per token,  1145.85 tokens per second)
llama_print_timings: prompt eval time =    3575.37 ms /   659 tokens (    5.43 ms per token,   184.32 tokens per second)
llama_print_timings:        eval time =     241.21 ms /     6 runs   (   40.20 ms per token,    24.87 tokens per second)
llama_print_timings:       total time =    4032.36 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.93 ms /     7 runs   (    0.56 ms per token,  1780.26 tokens per second)
llama_print_timings: prompt eval time =    3324.22 ms /   613 tokens (    5.42 ms per token,   184.40 tokens per second)
llama_print_timings:        eval time =     237.51 ms /     6 runs   (   39.59 ms per token,    25.26 tokens per second)
llama_print_timings:       to

In [27]:
score = 0
for i in range(len(results)):
  if results['answer'][i]['score'] == 'yes':
    score += 1
total_score_app5 = score/len(results)
print(f'Total score for Approach # 5 is {total_score_app5}')

Total score for Approach # 5 is 0.703125


In [28]:
app6 = pd.read_json('/content/drive/MyDrive/Thesis/retrievers/results_approach6.json')
len(app6)

66

In [29]:
results_list = []
for i in range(len(app6)):
  question = app6['question_name'][i]
  doc_txt = app6['context_l2_name'][i]['page_content']
  try:
    answer = retrieval_grader.invoke({"question": question, "document": doc_txt})
    results_list.append(pd.DataFrame({'question_name': question, 'answer': [answer]}))
  except JSONDecodeError as e:
    print(f"JSONDecodeError occurred for question: {question}. Skipping...")
    continue
  except OutputParserException as e:
    print(f"OutputParserException occurred for question: {question}. Skipping...")
    continue
results = pd.concat(results_list, ignore_index=True)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.25 ms /     6 runs   (    0.54 ms per token,  1845.59 tokens per second)
llama_print_timings: prompt eval time =    6068.09 ms /  1083 tokens (    5.60 ms per token,   178.47 tokens per second)
llama_print_timings:        eval time =     203.96 ms /     5 runs   (   40.79 ms per token,    24.52 tokens per second)
llama_print_timings:       total time =    6623.77 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.83 ms /     7 runs   (    0.55 ms per token,  1829.11 tokens per second)
llama_print_timings: prompt eval time =    5135.55 ms /   926 tokens (    5.55 ms per token,   180.31 tokens per second)
llama_print_timings:        eval time =     244.42 ms /     6 runs   (   40.74 ms per token,    24.55 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       3.40 ms /     6 runs   (    0.57 ms per token,  1764.19 tokens per second)
llama_print_timings: prompt eval time =    3389.78 ms /   629 tokens (    5.39 ms per token,   185.56 tokens per second)
llama_print_timings:        eval time =     196.47 ms /     5 runs   (   39.29 ms per token,    25.45 tokens per second)
llama_print_timings:       total time =    3786.40 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     437.95 ms
llama_print_timings:      sample time =       7.02 ms /     9 runs   (    0.78 ms per token,  1282.97 tokens per second)
llama_print_timings: prompt eval time =    3994.94 ms /   735 tokens (    5.44 ms per token,   183.98 tokens per second)
llama_print_timings:        eval time =     320.38 ms /     8 runs   (   40.05 ms per token,    24.97 tokens per second)
llama_print_timings:       total time =    4646.49 ms /   743 

In [31]:
score = 0
for i in range(len(results)):
  if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
    score += 1
total_score_app6 = score/len(results)
print(f'Total score for Approach # 6 is {total_score_app6}')

Total score for Approach # 6 is 0.6615384615384615


# Final results:

In [32]:
print(f'Total score for Approach # 1 is {total_score_app1}')
print(f'Total score for Approach # 2 is {total_score_app2}')
print(f'Total score for Approach # 3 is {total_score_app3}')
print(f'Total score for Approach # 4 is {total_score_app4}')
print(f'Total score for Approach # 5 is {total_score_app5}')
print(f'Total score for Approach # 6 is {total_score_app6}')

Total score for Approach # 1 is 0.703125
Total score for Approach # 2 is 0.6615384615384615
Total score for Approach # 3 is 0.703125
Total score for Approach # 4 is 0.6615384615384615
Total score for Approach # 5 is 0.703125
Total score for Approach # 6 is 0.6615384615384615
